In [88]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

data = pd.read_csv('data/cs-training.csv')

In [89]:
data.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [90]:
data.columns

Index(['Unnamed: 0', 'SeriousDlqin2yrs',
       'RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents'],
      dtype='object')

## Работа с пропущенными значениями

In [1]:
data.isna().any()

NameError: name 'data' is not defined

In [92]:
data['MonthlyIncome'].isna().value_counts()

False    120269
True      29731
Name: MonthlyIncome, dtype: int64

In [93]:
data = data.dropna()

In [94]:
data.isna().any()

Unnamed: 0                              False
SeriousDlqin2yrs                        False
RevolvingUtilizationOfUnsecuredLines    False
age                                     False
NumberOfTime30-59DaysPastDueNotWorse    False
DebtRatio                               False
MonthlyIncome                           False
NumberOfOpenCreditLinesAndLoans         False
NumberOfTimes90DaysLate                 False
NumberRealEstateLoansOrLines            False
NumberOfTime60-89DaysPastDueNotWorse    False
NumberOfDependents                      False
dtype: bool

In [95]:
data.shape

(120269, 12)

In [96]:
data.columns

Index(['Unnamed: 0', 'SeriousDlqin2yrs',
       'RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents'],
      dtype='object')

## Обучение KNN 

In [98]:
X, y = data.iloc[:, 2:], data.iloc[:, 1] 

In [99]:
X.columns

Index(['RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents'],
      dtype='object')

In [100]:
model = KNeighborsClassifier(n_neighbors=3)

In [101]:
from sklearn.model_selection import train_test_split

X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [107]:
model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [108]:
preds = model.predict(X_test)

In [110]:
preds.shape

(30068,)

In [114]:
y_test.shape

(30068,)

## Оценка качества модели

In [115]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, preds)

0.9255354529732606

## Получение одного предсказания 

In [56]:
# создадим массив a, состоящий из единичек. 
# Количество элементов в нём совпадает с количестовом столбцов в X 
# и этот массив можно рассматривать как показатели, предоставленные новым клиентом
a = np.ones((1, 10))

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [58]:
# делаем предсказание о том просрочит ли клиент этот кредит
pred_a = model.predict(a)

In [59]:
# модель говорит, что не просрочит
pred_a

array([0])

## Сохранение обученной модели в файл.
Чтобы можно было потом загрузить модель во flask

In [60]:
import pickle
filename = 'knn_fitted_model.sav'
pickle.dump(model, open(filename, 'wb'))


# НО!
После встраивания модели во flask выясняется, что она почему-то всегда выдаёт 0. Даже если ввести данные, по которым точно можно сказать, что клиент точно просрочит кредит. 

С чем это может быть связано? Давайте посмотрим на то сколько меток 1 и сколько меток 0 в данных, на которых модель училась.


In [62]:
y.value_counts()

0    111912
1      8357
Name: SeriousDlqin2yrs, dtype: int64

Оказывается данные очень несбалансированны! Нулей намного больше чем единиц и это сильно влияет на оценку качества модели.

Давайте представим такую ситуацию: Пусть существуют некоторая болезнь Х, заболеваемость которой составляет 1%. Предположим существуют тест на выявление этой болезни, который выглядит так : В 100% случаев, без исключения, выдавать результат, что человек не болен. Какова будет точность (accuracy) этого теста?

Ответ: 99%. Очень высокая точность, хотя этот тест полностью бесполезен.

В нашем случае мы наблюдаем очень похожую ситуацию. 

Как это можно исправить? Один из способ - сбалансировать датасет. Давайте возьмём все примеры с единицами и 10000 примеров с нулями. Тогда соотношение нулей и единичек будет практически равным

In [74]:
data_yes = data[data['SeriousDlqin2yrs'] == 1]
data_no = data[data['SeriousDlqin2yrs'] == 0].iloc[:10000,:]

In [75]:
data_new = data_yes.append(data_no)
data_new.shape

(18357, 12)

In [76]:
# снова сплитим датафрейм
X, y = data_new.iloc[:, 2:], data_new.iloc[:, 1] 
X_train,  X_test, y_train, y_test = train_test_split(X, y)

In [77]:
# обучаем KNN заново 
model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [78]:
# делаем предсказание на тестовом множестве
preds = model.predict(X_test)

In [79]:
# вычисляем accuracy 
accuracy_score(y_test, preds)

0.5738562091503268

57% !!! Это практически тоже самое, если бы предсказывали выдавать кредит или нет с помощью подбрасывания монетки. 
Давайте попробуем какой-нибудь другой алгоритм вместо KNN, может он покажет результаты лучше.

##  Используем другой алгоритм машинного обучения - "Случайный лес"
(На данный момент, необязательно знать как он работает)

In [82]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

Обучим модель и оценим качество. Модель даёт 75% точности. Намного лучше. Будем использовать именно эту модель в production.

In [83]:
model.fit(X_train, y_train)
preds = model.predict(X_test)
accuracy_score(y_test, preds)

0.7581699346405228

## Сохранение новой модели 

In [84]:
filename = 'rf_fitted_model.sav'
pickle.dump(model, open(filename, 'wb'))